In [2]:
import pandas as pd
import numpy as np
ApparelData = pd.read_csv('apparel-trainval.csv',sep=',',index_col = False)
from tqdm import tqdm 
newReconstructedData = pd.read_csv('ReconstructedApparelData.csv',sep=',',index_col = False)

In [113]:
UniqueLabels=np.array([0,1,2,3,4,5,6,7,8,9])
#UniqueLabels=np.array([0,1,2,3])
DataPerClass = []
index = 0
for i in UniqueLabels:
    tempdf = newReconstructedData.loc[newReconstructedData['label'] == i]
    DataPerClass.append(tempdf.sample(frac = 0.04))
    print(len(DataPerClass[index]))#+str(" for ")+str(i))
    index = index + 1
num_classes = len(UniqueLabels)
train = {}
val = {}
for i in range(0,num_classes):
    train[i] =  DataPerClass[i].sample(frac = 0.8)
    val[i]   =  DataPerClass[i].loc[~DataPerClass[i].index.isin(train[i].index)]
allTrain = []
allVal = [] 
for i in range(0,num_classes):
    allTrain.append(train[i])
    allVal.append(val[i])
train1 = pd.concat(allTrain, ignore_index= True)
val1 = pd.concat(allVal, ignore_index= True)
train1 = train1.sample(frac=1)
val1 = val1.sample(frac=1)
print(len(train1))
print(len(val1))
trainLabelVector = train1.values[:,0]
valLabelVector = val1.values[:,0]
C = 10
#m,n = train.values[:,1:].shape
X = train1.values[:,1:]
X.shape
fullXmatrix =[]
for i in tqdm(range(0,X.shape[0])):
    for p in range(0,num_classes):
        if(trainLabelVector[i]==p):
            #print('hey i am here')
            #pass
            tempArray = []
            for j in range(0,num_classes):
                for k in range(0,X.shape[1]):
                    tempArray.append(0.0)
                tempArray.append(0.0)
        else:     
            #print('hey i am here')
            tempArray = []
            for j in range(0,num_classes):
                if(j==trainLabelVector[i]):
                    for k in range(0,X.shape[1]):
                        tempArray.append(X[i][k])
                    tempArray.append(1.0)
                elif(j==p):
                    for k in range(0,X.shape[1]):
                        tempArray.append(-X[i][k])
                    tempArray.append(-1.0)                    
                else:
                    for k in range(0,X.shape[1]):
                        tempArray.append(0.0)
                    tempArray.append(0.0)
            fullXmatrix.append(np.array(tempArray))
fullXmatrix = np.array(fullXmatrix)
#print(trainLabelVector)
#print(fullXmatrix)
H = fullXmatrix@fullXmatrix.T
m = H.shape[0]
m
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
#print(np.vstack((np.eye(m)*-1,np.eye(m))))
#print(np.hstack((np.zeros(m), np.ones(m) * C)))
G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
Amatrix =[]
for i in tqdm(range(0,len(train1))):
    temp2 = [0.0]*m
    #temp2[int((i*num_classes)+trainLabelVector[i])] = 1.0
    Amatrix.append(temp2)
Amatrix = np.array(Amatrix)
#print(Amatrix)
A = cvxopt_matrix(Amatrix)
b = cvxopt_matrix(np.zeros(Amatrix.shape[0]))
q = cvxopt_matrix(-1*np.ones((m, 1)))
P = cvxopt_matrix(H)
opts = {'maxiters' : 30,'solver':'mosek'}
sol = cvxopt_solvers.qp(P, q, G, h ,options = opts)#, A, b,solver = 'glpk') , A, b
alphas2 = np.array(sol['x']) 
#print(alphas2)
w = fullXmatrix.T@alphas2
#print(w)
wForEachLabel = {}
for i in range(0,num_classes):
    wForEachLabel[i] = np.array_split(w,num_classes)[i]
predictedLabelVector = []
for i in tqdm(range(0,len(val1))):
    scoresForEachLabel = []
    for j in range(0,num_classes):
        score = (wForEachLabel[j].T@np.append(val1.values[i,1:],1.0).reshape(-1,1))
        scoresForEachLabel.append(score)
    maxScoreLabel = np.argmax(scoresForEachLabel)
    #print(scoresForEachLabel)
    #print(maxScoreLabel)
    predictedLabelVector.append(maxScoreLabel)
conMatrix = pd.crosstab(np.array(predictedLabelVector).ravel(),valLabelVector.reshape(-1,1).ravel())
conMatrix

KeyboardInterrupt: 

In [109]:
predictedLabelVector = []
for i in tqdm(range(0,len(train1))):
    scoresForEachLabel = []
    for j in range(0,num_classes):
        #score = (wForEachLabel[j].T@val1.values[i,1:].reshape(-1,1))
        score = (wForEachLabel[j].T@np.append(train1.values[i,1:],1.0).reshape(-1,1))
        scoresForEachLabel.append(score)
    maxScoreLabel = np.argmax(scoresForEachLabel)
    #print(scoresForEachLabel)
    #print(maxScoreLabel)
    predictedLabelVector.append(maxScoreLabel)
conMatrix = pd.crosstab(np.array(predictedLabelVector).ravel(),trainLabelVector.reshape(-1,1).ravel())
conMatrix

100%|████████████████████████████████████████| 960/960 [00:11<00:00, 86.81it/s]


col_0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
row_0,,,,,,,,,,
0,96,0,0,0,0,0,0,0,0,0
1,0,96,0,0,0,0,0,0,0,0
2,0,0,96,0,0,0,0,0,0,0
3,0,0,0,96,0,0,0,0,0,0
4,0,0,0,0,96,0,0,0,0,0
5,0,0,0,0,0,96,0,0,0,0
6,0,0,0,0,0,0,96,0,0,0
7,0,0,0,0,0,0,0,96,0,0
8,0,0,0,0,0,0,0,0,96,0


In [110]:
from sklearn import svm
clf = svm.SVC(gamma='scale', decision_function_shape='ovo',kernel='linear')
clf.fit(X, trainLabelVector) 
predictedLabelVector = []
for i in tqdm(range(0,len(val1))):
    predictedLabelVector.append(clf.predict(val1.values[i,1:].reshape(1,X.shape[1])))
conMatrix = pd.crosstab(np.array(predictedLabelVector).ravel(),valLabelVector.reshape(-1,1).ravel())
conMatrix

100%|██████████████████████████████████████| 240/240 [00:00<00:00, 3751.28it/s]


col_0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
row_0,,,,,,,,,,
0.0,18,0,0,0,0,1,1,0,0,0
1.0,1,22,0,4,1,0,0,0,0,0
2.0,1,1,17,0,3,0,9,0,0,0
3.0,0,1,1,18,2,0,2,0,0,0
4.0,0,0,5,0,15,0,2,0,0,0
5.0,0,0,0,0,0,21,0,3,1,2
6.0,4,0,1,1,3,0,10,0,0,0
7.0,0,0,0,0,0,1,0,20,0,2
8.0,0,0,0,1,0,0,0,0,23,0


In [111]:
lin_clf = svm.LinearSVC()
lin_clf.max_iter=10**5
lin_clf.fit(X, trainLabelVector) 
predictedLabelVector = []
for i in tqdm(range(0,len(val1))):
    predictedLabelVector.append(lin_clf.predict(val1.values[i,1:].reshape(1,X.shape[1])))
conMatrix = pd.crosstab(np.array(predictedLabelVector).ravel(),valLabelVector.reshape(-1,1).ravel())
conMatrix

100%|██████████████████████████████████████| 240/240 [00:00<00:00, 6317.40it/s]


col_0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
row_0,,,,,,,,,,
0.0,14,3,1,0,0,0,2,0,3,0
1.0,1,19,0,2,0,1,0,1,0,0
2.0,2,0,13,0,4,0,5,1,0,0
3.0,1,1,0,19,1,0,4,0,0,0
4.0,0,0,7,0,16,0,2,0,0,0
5.0,2,1,1,1,0,19,1,2,0,1
6.0,4,0,2,0,3,0,10,0,0,1
7.0,0,0,0,0,0,2,0,19,2,2
8.0,0,0,0,1,0,1,0,0,18,0


In [80]:
x = np.arange(8.0)
np.array_split(x, 2)

[array([0., 1., 2., 3.]), array([4., 5., 6., 7.])]